In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np 
from dotenv import load_dotenv
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
from sklearn.metrics import classification_report

In [3]:
load_dotenv()

True

In [4]:
# Spotify API credentials
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

# Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [5]:
# Read in the data
data = pd.read_csv('../Data/song_data(pop-artist).csv')
df = pd.DataFrame(data)
df

,Unnamed: 0,id,title,all_artists,popularity_artist,popularity_song,release_date,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,7zgqtptZvhf8GEmdsM2vp2,...Ready For It?,Taylor Swift,92,0,2017-09-03,0.615,0.779,2,-6.454,1,0.06650,0.000000,0.1550,0.453,160.000,208198,4
1,1,4Vxu50qVrQcycjRyJQaZLC,Life Changes,Thomas Rhett,74,63,2017-09-08,0.687,0.845,7,-4.370,1,0.10000,0.000000,0.0452,0.809,87.972,190227,4
2,2,6b8Be6ljOzmkOmFslEb23P,24K Magic,Bruno Mars,87,78,2016-11-17,0.818,0.803,1,-4.282,1,0.03400,0.000000,0.1530,0.632,106.970,225983,4
3,3,0afhq8XCExXpqazXczTSve,Galway Girl,Ed Sheeran,91,77,2017-03-03,0.624,0.876,9,-3.374,1,0.07350,0.000000,0.3270,0.781,99.943,170827,4
4,4,1HNkqx9Ahdgi1Ixy2xkKkL,Photograph,Ed Sheeran,91,83,2014-06-21,0.614,0.379,4,-10.480,1,0.60700,0.000464,0.0986,0.201,107.989,258987,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9918,9918,4UFlPCB4THnQ9TlPHqIQow,Funeral For A Friend / Love Lies Bleeding,Elton John,82,0,1973-10-05,0.410,0.761,9,-8.507,0,0.01980,0.084700,0.2470,0.193,138.712,666572,4
9919,9919,5pSSEkT0963muzzIjsVkrs,Fool's Overture,Supertramp,67,52,1977-01-01,0.406,0.306,3,-10.482,1,0.31300,0.007900,0.0727,0.073,135.272,652560,4
9920,9920,7gC6Rbllqf1yXNC02e5jz2,Heart of the Sunrise - 2003 Remaster,Yes,59,45,1971-11-26,0.362,0.507,1,-11.229,1,0.01740,0.216000,0.1130,0.456,146.641,634440,3
9921,9921,6Ff77WXC58MkhLE5A1qgY1,Venus And Mars / Rock Show / Jet - Live / Rema...,Wings,69,0,1976-12-10,0.331,0.733,2,-8.671,1,0.08870,0.001740,0.9470,0.380,128.512,620747,4


In [6]:
# remove records where popularity_song is 0
df = df[df.popularity_song > 0]
df

,Unnamed: 0,id,title,all_artists,popularity_artist,popularity_song,release_date,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
1,1,4Vxu50qVrQcycjRyJQaZLC,Life Changes,Thomas Rhett,74,63,2017-09-08,0.687,0.845,7,-4.370,1,0.1000,0.000000,0.0452,0.809,87.972,190227,4
2,2,6b8Be6ljOzmkOmFslEb23P,24K Magic,Bruno Mars,87,78,2016-11-17,0.818,0.803,1,-4.282,1,0.0340,0.000000,0.1530,0.632,106.970,225983,4
3,3,0afhq8XCExXpqazXczTSve,Galway Girl,Ed Sheeran,91,77,2017-03-03,0.624,0.876,9,-3.374,1,0.0735,0.000000,0.3270,0.781,99.943,170827,4
4,4,1HNkqx9Ahdgi1Ixy2xkKkL,Photograph,Ed Sheeran,91,83,2014-06-21,0.614,0.379,4,-10.480,1,0.6070,0.000464,0.0986,0.201,107.989,258987,4
5,5,0ofbQMrRDsUaVKq2mGLEAb,Havana (feat. Young Thug),Young Thug,83,1,2017-08-03,0.768,0.517,7,-4.323,0,0.1860,0.000038,0.1040,0.418,104.992,216897,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9915,9915,7odcDCEpJdzycpucqGK3mj,"Remember the Future, Pt. 2",Nektar,31,18,1973-11-13,0.435,0.583,4,-12.894,0,0.4780,0.114000,0.1120,0.347,109.866,1140973,4
9916,9916,4OjKHySJHstsImlUW3qfml,Alice's Restaurant Massacree,Arlo Guthrie,44,48,1967-08-28,0.561,0.366,9,-14.140,1,0.8510,0.000000,0.8130,0.646,78.659,1116933,4
9917,9917,7xBXbeQkcVeOELZgGa705e,"Remember the Future, Pt. 1",Nektar,31,25,1973-11-13,0.391,0.590,9,-13.622,1,0.6760,0.553000,0.0661,0.479,113.675,1001120,4
9919,9919,5pSSEkT0963muzzIjsVkrs,Fool's Overture,Supertramp,67,52,1977-01-01,0.406,0.306,3,-10.482,1,0.3130,0.007900,0.0727,0.073,135.272,652560,4


In [7]:
# check NA values from title
df['title'].isnull().sum()

0

In [11]:
# Drop unnecessary columns; unnamed
df = df.drop(columns=['Unnamed: 0'])
df.head()

,id,title,all_artists,popularity_artist,popularity_song,release_date,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
1,4Vxu50qVrQcycjRyJQaZLC,Life Changes,Thomas Rhett,74,63,2017-09-08,0.687,0.845,7,-4.370,1,0.1000,0.000000,0.0452,0.809,87.972,190227,4
2,6b8Be6ljOzmkOmFslEb23P,24K Magic,Bruno Mars,87,78,2016-11-17,0.818,0.803,1,-4.282,1,0.0340,0.000000,0.1530,0.632,106.970,225983,4
3,0afhq8XCExXpqazXczTSve,Galway Girl,Ed Sheeran,91,77,2017-03-03,0.624,0.876,9,-3.374,1,0.0735,0.000000,0.3270,0.781,99.943,170827,4
4,1HNkqx9Ahdgi1Ixy2xkKkL,Photograph,Ed Sheeran,91,83,2014-06-21,0.614,0.379,4,-10.480,1,0.6070,0.000464,0.0986,0.201,107.989,258987,4
5,0ofbQMrRDsUaVKq2mGLEAb,Havana (feat. Young Thug),Young Thug,83,1,2017-08-03,0.768,0.517,7,-4.323,0,0.1860,0.000038,0.1040,0.418,104.992,216897,4


In [12]:
# Create a list of all the artists
artists = df['all_artists'].tolist()

In [13]:
# Describe the features
df_describe = df.describe()
df_describe.head(10)

,popularity_artist,popularity_song,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,6600.000000,6600.000000,6600.000000,6600.000000,6600.000000,6600.000000,6600.000000,6600.000000,6600.000000,6600.000000,6600.000000,6600.000000,6.600000e+03,6600.000000
mean,63.080152,48.516364,0.571883,0.666534,5.246818,-7.366078,0.664697,0.219285,0.083560,0.186554,0.484610,120.547484,2.804095e+05,3.942879
std,17.238109,19.613890,0.165417,0.219595,3.573931,4.322217,0.472132,0.276974,0.228501,0.155939,0.246824,28.266113,3.689502e+05,0.348065
min,0.000000,1.000000,0.000000,0.000000,0.000000,-44.907000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.001720e+05,0.000000
25%,55.000000,37.000000,0.469000,0.536000,2.000000,-8.584250,0.000000,0.013400,0.000000,0.093600,0.289000,99.928750,1.975422e+05,4.000000
50%,66.000000,51.000000,0.585000,0.712000,5.000000,-6.239000,1.000000,0.083900,0.000014,0.123000,0.481000,119.974000,2.238800e+05,4.000000
75%,75.000000,64.000000,0.689000,0.838000,9.000000,-4.744000,1.000000,0.334250,0.003775,0.229000,0.677000,138.179500,2.594130e+05,4.000000
max,100.000000,88.000000,0.980000,1.000000,11.000000,-0.716000,1.000000,0.995000,0.999000,0.981000,0.981000,214.419000,6.000295e+06,5.000000


In [14]:
# Create a copy of the main dataframe df and perform all data cleansing operations on copied dataframe
df_copy = df.copy()

In [15]:
popularity_threshold = df_describe.iloc[6:7,1:2]
print(popularity_threshold)

     popularity_song
75%             64.0


In [16]:
# add new column hit_prediction to the df_copy dataframe and fill values for each song to be 1 (represents a hit) OR to be 0 (represents a not hit). This is done by checking popularity is greater than OR leass than 57.
prediction = []
for value in df_copy['popularity_song']:
    if value >= popularity_threshold.values:
#        print(f"If loop: {value}")
        prediction.append(1)
    else:
#        print(f"Else loop: {value}")
        prediction.append(0)
df_copy['hit_prediction'] = prediction

In [18]:
df_copy = df_copy.copy()
df_copy = df_copy_split.drop(columns=['all_artists','id','title','release_date','key','mode','duration_ms','time_signature'])
df_copy

,popularity_artist,popularity_song,danceability,energy,loudness,acousticness,instrumentalness,liveness,valence,tempo,hit_prediction
1,74,63,0.687,0.845,-4.370,0.1000,0.000000,0.0452,0.809,87.972,0
2,87,78,0.818,0.803,-4.282,0.0340,0.000000,0.1530,0.632,106.970,1
3,91,77,0.624,0.876,-3.374,0.0735,0.000000,0.3270,0.781,99.943,1
4,91,83,0.614,0.379,-10.480,0.6070,0.000464,0.0986,0.201,107.989,1
5,83,1,0.768,0.517,-4.323,0.1860,0.000038,0.1040,0.418,104.992,0
...,...,...,...,...,...,...,...,...,...,...,...
9915,31,18,0.435,0.583,-12.894,0.4780,0.114000,0.1120,0.347,109.866,0
9916,44,48,0.561,0.366,-14.140,0.8510,0.000000,0.8130,0.646,78.659,0
9917,31,25,0.391,0.590,-13.622,0.6760,0.553000,0.0661,0.479,113.675,0
9919,67,52,0.406,0.306,-10.482,0.3130,0.007900,0.0727,0.073,135.272,0


In [20]:
# Target
y = df_copy['hit_prediction']

# Features
X = df_copy.iloc[:,:-1]

In [48]:
# Creating training, validation, and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=10)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state=10)

In [49]:
#logistic regression model no scaler
model2 = LogisticRegression()
model2.fit(x_train, y_train)
predictions2 = model2.predict(x_test)

C:\Users\dhaam\anaconda3\envs\dev\envs\dev\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [50]:
print(f"Training Data Score: {model2.score(x_train, y_train)}")
print(f"Testing Data Score: {model2.score(x_test, y_test)}")

Training Data Score: 0.9867995689655172
Testing Data Score: 0.9872727272727273


In [51]:
#using standard scaler
x_scaler = StandardScaler().fit(x_train)

x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

#logreg model scaled
model_LR = LogisticRegression()
model_LR.fit(x_train_scaled, y_train)

LogisticRegression()

In [52]:
print(f"Training Data Score: {model_LR.score(x_train_scaled, y_train)}")
print(f"Testing Data Score: {model_LR.score(x_test_scaled, y_test)}")

Training Data Score: 0.9997306034482759
Testing Data Score: 0.996969696969697


In [59]:
#using MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

x_scaler2 = MinMaxScaler().fit(x_train)

x_train_scaled2 = x_scaler2.transform(x_train)
x_test_scaled2 = x_scaler2.transform(x_test)

#logreg model minmax scaled
model_LR2 = LogisticRegression()
model_LR2.fit(x_train_scaled2, y_train)

LogisticRegression()

In [61]:
print(f"Training Data Score: {model_LR2.score(x_train_scaled2, y_train)}")
print(f"Testing Data Score: {model_LR2.score(x_test_scaled2, y_test)}")

Training Data Score: 0.9857219827586207
Testing Data Score: 0.9787878787878788


In [69]:
# Calculating the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
lr_pred = model2.predict(x_test)
print(confusion_matrix(y_test, lr_pred))
print('\n')
print(classification_report(y_test, lr_pred))


[[1196   15]
 [   6  433]]


              precision    recall  f1-score   support

           0       1.00      0.99      0.99      1211
           1       0.97      0.99      0.98       439

    accuracy                           0.99      1650
   macro avg       0.98      0.99      0.98      1650
weighted avg       0.99      0.99      0.99      1650

